# Project Phase 3 - Machine Learning (Tf-idf)

In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')


In [2]:
path = 'newest dcbox resource/'
dest_path = 'newest dcbox resource/'
dirs = os.listdir(path)
dirs.remove('.ipynb_checkpoints')

In [3]:
df_train = pd.DataFrame()
# body_df['sentences']
counter = 0
for file in dirs:
    body_df = pd.read_csv(path + file)
    body_df.loc[:, 'category'] = file.replace('.json.csv','')
    body_df.loc[:, 'category_id'] = counter
    df_train = df_train.append(body_df,ignore_index = True)
    counter += 1


In [4]:
features = tfidf.fit_transform(df_train.sentences).toarray()
labels = df_train.category_id
features.shape


(39366, 32457)

In [5]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
category_id_df = df_train[['category', 'category_id']]
category_to_id = dict(category_id_df.values)

for category, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(category))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    

# 'business':
  . Most correlated unigrams:
. mlb
. elton
  . Most correlated bigrams:
. league teams
. minor league
# 'children education':
  . Most correlated unigrams:
. brothman
. cadca
  . Most correlated bigrams:
. winthrop harbor
. state union
# 'consumer':
  . Most correlated unigrams:
. consumers
. spectrum
  . Most correlated bigrams:
. traced act
. 2019 house
# 'fashion beauty':
  . Most correlated unigrams:
. collins
. beautiful
  . Most correlated bigrams:
. west virginians
. senator collins
# 'fitness sports':
  . Most correlated unigrams:
. que
. sports
  . Most correlated bigrams:
. professor myers
. perdue said
# 'food beverage':
  . Most correlated unigrams:
. thanksgiving
. food
  . Most correlated bigrams:
. increase year
. native hawaiian
# 'green cleantech':
  . Most correlated unigrams:
. corina
. green
  . Most correlated bigrams:
. al green
. impeachment inquiries
# 'healthcare':
  . Most correlated unigrams:
. risch
. healthcare
  . Most correlated bigrams:
. 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df_train['sentences'], df_train['category'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [7]:
print(clf.predict(count_vect.transform(["My cousin has two cats and one dog."])))

['pet']


In [8]:
print(clf.predict(count_vect.transform([
    "I have transfered from Pasadena city College to University of California, Riverside."])))

['software tech']


In [9]:
print(clf.predict(count_vect.transform(["I am hungry, so I am going to buy something to eat."])))

['fashion beauty']


In [10]:
print(clf.predict(count_vect.transform([""])))

['consumer']


In [11]:
Y_pred = clf.predict(count_vect.transform(X_test))

In [12]:
Y_pred_self = clf.predict(X_train_tfidf)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,Y_pred)

0.15139199349725665